In [1]:
import pandas as pd
import json

# 데이터 불러오기 및 전처리
df1 = pd.read_csv('data/교통사고 데이터/전국교통사고다발지역(2012-2021).csv', encoding='euc-kr')
# 2013-2016년 데이터만 추출
df1 = df1[(df1['사고연도'] >= 2014) & (df1['사고연도'] <= 2016)]

drop_list1 = ['사고연도', '사고유형구분', '데이터기준일자', '제공기관코드', '제공기관명']
df1 = df1.drop(columns=drop_list1)

df2 = pd.read_csv('data/교통사고 데이터/전국교통사고다발지역(2017-2023).csv', encoding='euc-kr')
drop_list2 = ['법정동코드']
df2 = df2.drop(columns=drop_list2)

df = pd.concat([df1, df2])

# '위치코드'의 앞 5자리 숫자로 그룹화하고 사고건수를 합산
df['위치코드_시군구'] = df['위치코드'].astype(str).str[:5]

# 그룹화하고 첫 번째 '사고지역위치명'의 처음 두 단어를 사용
df_grouped = df.groupby('위치코드_시군구', as_index=False).agg({
    '사고건수': 'sum',
    '사고지역위치명': lambda x: ' '.join(x.iloc[0].split()[:2])  # 첫 번째 위치명에서 처음 두 단어 추출
})
df_grouped = df_grouped.rename(columns={'사고지역위치명': '시군구명'})

# 결과를 JSON 파일로 저장
df_grouped.to_json('data/accident_data.json', orient='records', force_ascii=False)
print(df.shape)

(9254, 15)


In [38]:
df_sorted = df_grouped.sort_values('사고건수', ascending=False)

# 결과를 JSON 파일로 저장 (표2 표시용)
df_sorted.head(10).to_json('data/top10_accident_grouped.json', orient='records', force_ascii=False)

# 결과 출력
print(df_sorted)

    위치코드_시군구  사고건수          시군구명
46     11560  1601    서울특별시 영등포구
51     11710  1517     서울특별시 송파구
188    27290  1412     대구광역시 달서구
22     11230  1399    서울특별시 동대문구
216    30170  1393      대전광역시 서구
..       ...   ...           ...
128    20234     4      경상남도 창원시
67     13074     4       경기도 파주시
66     13064     4      경기도 의정부시
207    28720     4     인천광역시 옹진군
133    21035     3  제주특별자치도 서귀포시

[447 rows x 3 columns]


In [39]:
import pandas as pd
import re

# 주소명을 통일하는 함수
def standardize_address(address):
    address = re.sub(r'강원 ', '강원도 ', address)
    address = re.sub(r'경북 ', '경상북도 ', address)
    address = re.sub(r'광주 ', '광주광역시 ', address)
    address = re.sub(r'대구 ', '대구광역시 ', address)
    address = re.sub(r'인천 ', '인천광역시 ', address)
    address = re.sub(r'전북 ', '전라북도 ', address)
    address = re.sub(r'충남 ', '충청남도 ', address)
    # 필요에 따라 다른 표준화 작업 추가 가능
    return address.strip()

# 표준화된 주소명을 만들기
df['사고지역위치명'] = df['사고지역위치명'].apply(standardize_address)

# 위치명으로 그룹화하고 사고건수를 합산
df_grouped_1 = df.groupby('사고지역위치명', as_index=False).agg({
    '사고건수': 'sum'
})

# 사고건수 기준으로 내림차순 정렬
df_sorted_1 = df_grouped_1.sort_values('사고건수', ascending=False)

# 결과를 JSON 파일로 저장 (표1 표시용)
df_sorted_1.head(10).to_json('data/top10_accident_locations.json', orient='records', force_ascii=False)


In [41]:
# '사고지역위치명'에서 처음 두 단어 추출 함수
def extract_first_two_words(location):
    return ' '.join(location.split()[:2])

# 처음 두 단어로 새로운 열 생성
df['시군구명'] = df['사고지역위치명'].apply(extract_first_two_words)

# '시군구명'로 그룹화하고 사고건수를 합산
df_grouped_2 = df.groupby('시군구명', as_index=False).agg({
    '사고건수': 'sum'
})

# df_sorted_2 = df_grouped_2.sort_values('사고건수', ascending=False)

# # 결과를 JSON 파일로 저장 (표2 표시용)
# df_sorted_2.head(10).to_json('data/top10_accident_grouped.json', orient='records', force_ascii=False)

# # 결과 출력
# print(df_sorted_2)

In [42]:
import pandas as pd
import json

# '시군구명'에서 시도 이름 추출 함수
def extract_city_name(region_name):
    return region_name.split()[0]

# 시도 이름으로 새로운 열 생성
df['시도명'] = df['시군구명'].apply(extract_city_name)

# 시도명으로 그룹화하고 사고건수를 합산
df_grouped = df.groupby('시도명', as_index=False).agg({
    '사고건수': 'sum'
})

# 각 시도별로 데이터를 분류하여 JSON 파일로 저장
city_data = {}
for _, row in df_grouped.iterrows():
    city_name = row['시도명']
    accident_count = row['사고건수']
    city_data[city_name] = accident_count

    # 파일 저장 경로 설정
    file_name = f'data/시도별json/{city_name}_accident_data.json'
    with open(file_name, 'w', encoding='utf-8') as file:
        json.dump({city_name: accident_count}, file, ensure_ascii=False, indent=4)

print("JSON 파일이 성공적으로 생성되었습니다.")


JSON 파일이 성공적으로 생성되었습니다.


In [3]:
import pandas as pd
import json

# 데이터를 DataFrame으로 변환
df1 = pd.read_json('data/시도별 json 파일/강원_accident_data.json')
df2 = pd.read_json('data/시도별 json 파일/강원도_accident_data.json')

# 두 DataFrame을 결합
combined_df = pd.concat([df1, df2])

# '시군구명' 컬럼에서 도시 이름 추출 및 사고건수 집계
combined_df['도시명'] = combined_df['시군구명'].apply(lambda x: x.split(' ')[1])
result_df = combined_df.groupby('도시명')['사고건수'].sum().reset_index()

# 결과 DataFrame을 JSON 파일로 내보내기
result_df.to_json('data/시도별 json 파일/강원도_accidents.json', orient='records', force_ascii=False)

# 결과 확인
print(result_df)

    도시명  사고건수
0   강릉시   352
1   고성군    98
2   동해시   243
3   삼척시   222
4   속초시   349
5   양구군    78
6   양양군   115
7   영월군   124
8   원주시   914
9   인제군    74
10  정선군    93
11  철원군    94
12  춘천시   442
13  태백시   192
14  평창군   102
15  홍천군   107
16  화천군    43
17  횡성군   152


In [4]:
import pandas as pd
import json

# 데이터를 DataFrame으로 변환
df1 = pd.read_json('data/시도별 json 파일/경북_accident_data.json')
df2 = pd.read_json('data/시도별 json 파일/경상북도_accident_data.json')

# 두 DataFrame을 결합
combined_df = pd.concat([df1, df2])

# '시군구명' 컬럼에서 도시 이름 추출 및 사고건수 집계
combined_df['도시명'] = combined_df['시군구명'].apply(lambda x: x.split(' ')[1])
result_df = combined_df.groupby('도시명')['사고건수'].sum().reset_index()

# 결과 DataFrame을 JSON 파일로 내보내기
result_df.to_json('data/시도별 json 파일/경상북도_accidents.json', orient='records', force_ascii=False)

# 결과 확인
print(result_df)

    도시명  사고건수
0   경산시   475
1   경주시   530
2   고령군   100
3   구미시   500
4   군위군    30
5   김천시   360
6   문경시   218
7   봉화군    58
8   상주시   314
9   성주군   173
10  안동시   483
11  영덕군   177
12  영양군    96
13  영주시   369
14  영천시   265
15  예천군   127
16  울릉군     8
17  울진군    40
18  의성군   130
19  청도군   121
20  청송군    36
21  칠곡군   309
22  포항시   748


In [9]:
import pandas as pd
import json

# 데이터를 DataFrame으로 변환
df1 = pd.read_json('data/시도별 json 파일/충청남도_accident_data.json')
df2 = pd.read_json('data/시도별 json 파일/충남_accident_data.json')

# 두 DataFrame을 결합
combined_df = pd.concat([df1, df2])

# '시군구명' 컬럼에서 도시 이름 추출 및 사고건수 집계
combined_df['도시명'] = combined_df['시군구명'].apply(lambda x: x.split(' ')[1])
result_df = combined_df.groupby('도시명')['사고건수'].sum().reset_index()

# 결과 DataFrame을 JSON 파일로 내보내기
result_df.to_json('data/시도별 json 파일/충청남도_accidents.json', orient='records', force_ascii=False)

# 결과 확인
print(result_df)

    도시명  사고건수
0   계룡시    64
1   공주시   276
2   금산군   119
3   논산시   235
4   당진시   316
5   보령시   296
6   부여군   144
7   서산시   340
8   서천군   178
9   아산시   457
10  예산군   149
11  천안시   893
12  청양군    94
13  태안군   172
14  홍성군   201


In [12]:
# 데이터를 DataFrame으로 변환
combined_df = pd.read_json('data/시도별 json 파일/서울특별시_accidents.json')

# '시군구명' 컬럼에서 도시 이름 추출 및 사고건수 집계
combined_df['도시명'] = combined_df['시군구명'].apply(lambda x: x.split(' ')[1])
result_df = combined_df.groupby('도시명')['사고건수'].sum().reset_index()

# 결과 DataFrame을 JSON 파일로 내보내기
result_df.to_json('data/시도별 json 파일/서울_accidents.json', orient='records', force_ascii=False)

# 결과 확인
print(result_df)

     도시명  사고건수
0    강남구  1219
1    강동구  1037
2    강북구  1419
3    강서구  1073
4    관악구  1352
5    광진구   896
6    구로구  1086
7    금천구   816
8    노원구  1064
9    도봉구   740
10  동대문구  1421
11   동작구  1091
12   마포구   982
13  서대문구   768
14   서초구  1117
15   성동구   930
16   성북구  1312
17   송파구  1536
18   양천구  1056
19  영등포구  1683
20   용산구   805
21   은평구   994
22   종로구   966
23    중구   990
24   중랑구  1118


## Part 2.

In [83]:
# 여러 연도의 데이터를 결합하기 위해 파일 경로 목록을 만듭니다.
file_paths = [
    'data/교통사고 데이터/도로교통공단_일자별 시군구별 교통사고 건수(2016).csv',
    'data/교통사고 데이터/도로교통공단_일자별 시군구별 교통사고 건수(2017).csv',
    'data/교통사고 데이터/도로교통공단_일자별 시군구별 교통사고 건수(2018).csv',
    'data/교통사고 데이터/도로교통공단_일자별 시군구별 교통사고 건수(2019).csv',
    'data/교통사고 데이터/도로교통공단_일자별 시군구별 교통사고 건수(2020).csv',
    'data/교통사고 데이터/도로교통공단_일자별 시군구별 교통사고 건수(2021).csv',
    'data/교통사고 데이터/도로교통공단_일자별 시군구별 교통사고 건수(2022).csv',
    'data/교통사고 데이터/도로교통공단_일자별 시군구별 교통사고 건수(2023).csv'
]

# 데이터를 저장할 빈 리스트를 만듭니다.
df3_list = []

# 각 파일을 불러와서 리스트에 추가합니다.
for file_path in file_paths:
    year = int(file_path[-9:-5])  # 파일명에서 연도 추출
    df3 = pd.read_csv(file_path, encoding='euc-kr')
    df3['발생년도'] = year  # 발생년도 열 추가
    df3_list.append(df3)

# 모든 데이터를 하나의 데이터프레임으로 결합합니다.
df3_combined = pd.concat(df3_list, ignore_index=True)


In [85]:
# ----------------------------
# 어린이날 교통사고 추이 분석
# ----------------------------

# 어린이날 (5월 5일) 데이터 필터링
df_children_day = df3_combined[(df3_combined['발생월'] == 5) & (df3_combined['발생일'] == 5)]

# 연도별 사고 건수 및 사망자수, 중상자수, 경상자수 집계
children_day_stats = df_children_day.groupby('발생년도').sum()[['사고건수', '사망자수', '중상자수', '경상자수']].reset_index()

children_day_stats.to_json('data/55_accident.json', orient='records', force_ascii=False)

In [99]:
# 연도별 설날 연휴 날짜 딕셔너리
lunar_new_year_dates = {
    2016: [('02-07'), ('02-08'), ('02-09'), ('02-10')],
    2017: [('01-27'), ('01-28'), ('01-29'), ('01-30')],
    2018: [('02-15'), ('02-16'), ('02-17'), ('02-18')],
    2019: [('02-02'), ('02-03'), ('02-04'), ('02-05'), ('02-06')],
    2020: [('01-24'), ('01-25'), ('01-26'), ('01-27')],
    2021: [('02-11'), ('02-12'), ('02-13'), ('02-14')],
    2022: [('01-31'), ('02-01'), ('02-02')],
    2023: [('01-21'), ('01-22'), ('01-23'), ('01-24')]
}

# 연도별 추석 연휴 날짜 딕셔너리
chuseok_dates = {
    2016: [('09-14'), ('09-15'), ('09-16')],
    2017: [('10-03'), ('10-04'), ('10-05'), ('10-06')],
    2018: [('09-23'), ('09-24'), ('09-25'), ('09-26')],
    2019: [('09-12'), ('09-13'), ('09-14')],
    2020: [('09-30'), ('10-01'), ('10-02'), ('10-03')],
    2021: [('09-20'), ('09-21'), ('09-22')],
    2022: [('09-09'), ('09-10'), ('09-11'), ('09-12')],
    2023: [('09-28'), ('09-29'), ('09-30')]
}

def filter_holiday_data(df, holiday_dates, holiday_name):
    holiday_data_list = []
    for year, dates in holiday_dates.items():
        # 휴일 기간 계산
        start_date = dates[0].replace('-', '/')
        end_date = dates[-1].replace('-', '/')
        holiday_period = f"{start_date}~{end_date}"
        
        for date_str in dates:
            month, day = map(int, date_str.split('-'))
            filtered_data = df[
                (df['발생년도'] == year) &
                (df['발생월'] == month) &
                (df['발생일'] == day)
            ].copy()  # 데이터프레임의 복사본을 생성하여 열을 추가함
            filtered_data['휴일명'] = holiday_name
            filtered_data['휴일기간'] = holiday_period  # 휴일 기간을 추가
            holiday_data_list.append(filtered_data)
    return pd.concat(holiday_data_list, ignore_index=True)

# 설날 교통사고 데이터 필터링 및 집계
df_lunar_new_year = filter_holiday_data(df3_combined, lunar_new_year_dates, '설날')
lunar_new_year_stats = df_lunar_new_year.groupby('발생년도').sum()[['사고건수', '사망자수', '중상자수', '경상자수']].reset_index()

# 추석 교통사고 데이터 필터링 및 집계
df_chuseok = filter_holiday_data(df3_combined, chuseok_dates, '추석')
chuseok_stats = df_chuseok.groupby('발생년도').sum()[['사고건수', '사망자수', '중상자수', '경상자수']].reset_index()

df_lunar_new_year.to_json("data/luner_new_year_accident.json", orient='records', force_ascii=False)
df_chuseok.to_json("data/chuseok_accident.json", orient='records', force_ascii=False)

# 추가된 휴일명 및 휴일기간 열을 포함한 데이터프레임 확인
print(df_lunar_new_year)
print(df_chuseok)


      발생월  발생일  시도   시군구  사고건수  사망자수  중상자수  경상자수  부상신고자수  발생년도 휴일명  \
0       2    7  서울   용산구     3     0     1     2       0  2016  설날   
1       2    7  서울  동대문구     4     0     1     3       0  2016  설날   
2       2    7  서울   성북구     1     0     0     0       1  2016  설날   
3       2    7  서울   도봉구     2     0     1     1       0  2016  설날   
4       2    7  서울   은평구     2     0     1     1       0  2016  설날   
...   ...  ...  ..   ...   ...   ...   ...   ...     ...   ...  ..   
4603    1   24  대전   대덕구     1     0     0     1       0  2023  설날   
4604    1   24  울산    중구     1     0     1     0       0  2023  설날   
4605    1   24  울산    남구     3     0     2     1       0  2023  설날   
4606    1   24  울산    북구     1     0     1     0       0  2023  설날   
4607    1   24  울산   울주군     3     0     1     5       0  2023  설날   

             휴일기간  
0     02/07~02/10  
1     02/07~02/10  
2     02/07~02/10  
3     02/07~02/10  
4     02/07~02/10  
...           ...  
4603  01/21~01/24  

In [96]:
# 월드컵 기간 설정 (예시로 대한민국 경기 날짜 포함)
world_cup_dates_2018 = [
    ('06-14', '07-15')  # 2018 러시아 월드컵 전체 기간
]

world_cup_dates_2022 = [
    ('11-20', '12-18')  # 2022 카타르 월드컵 전체 기간
]

def filter_world_cup_data(df, world_cup_dates, year):
    world_cup_data_list = []
    for start_date, end_date in world_cup_dates:
        start_month, start_day = map(int, start_date.split('-'))
        end_month, end_day = map(int, end_date.split('-'))
        
        filtered_data = df[
            (df['발생년도'] == year) &
            ((df['발생월'] == start_month) & (df['발생일'] >= start_day) | (df['발생월'] == end_month) & (df['발생일'] <= end_day))
        ]
        world_cup_data_list.append(filtered_data)
    
    return pd.concat(world_cup_data_list, ignore_index=True)

# 2018년 데이터 필터링
df_wc_2018 = filter_world_cup_data(df3_combined, world_cup_dates_2018, 2018)

# 2022년 데이터 필터링
df_wc_2022 = filter_world_cup_data(df3_combined, world_cup_dates_2022, 2022)

# 연도별 사고 건수 집계
wc_2018_stats = df_wc_2018.groupby('발생년도').sum()[['사고건수']].reset_index()
wc_2022_stats = df_wc_2022.groupby('발생년도').sum()[['사고건수']].reset_index()

# 두 기간의 데이터 병합
wc_stats_combined = pd.concat([wc_2018_stats, wc_2022_stats], ignore_index=True)

wc_stats_combined.to_json("data/worldcup.json", orient='records', force_ascii=False)
